## What is this?

This Jupyter Notebook contains Python code for building a LSTM Recurrent Neural Network to train a simple Part-of-Speech Tagger.

This code is supplied without license, warranty or support. Feel free to do with it what you will.


## Data Import

In [1]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown.words()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [13]:
def posmapper(postag):
  s = postag[0] 
  if s in ['N','V','A']:
    return s
  elif s == 'J':
    return 'R'
  else:
    return 'O'

training_data = []
#    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])

word_to_ix = {}
#a = 10

for sent in brown.tagged_sents():
  words = []
  poss = []
  for word, pos in sent:
    words.append(word)
    poss.append(posmapper(pos))
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)
  training_data.append((words,poss))
#  a-=1
#  if a < 1:
#    break
#print(word_to_ix)
#print(training_data[0])

tag_to_ix = {"N": 0, "A": 1, "V": 2, "R": 3, "O": 4}

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).cuda()


# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 100
HIDDEN_DIM = 100

######################################################################
# Create the model:


class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

######################################################################
# Train the model:


model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix)).cuda()
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        print("loss={:6.5f}".format(loss))


# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)


Streaming output truncated to the last 5000 lines.
loss=0.19301
loss=0.42232
loss=0.38018
loss=0.13259
loss=0.36870
loss=0.21358
loss=0.32226
loss=0.15029
loss=0.26793
loss=0.22129
loss=0.20314
loss=0.14368
loss=0.22349
loss=0.31977
loss=0.18610
loss=0.22019
loss=0.23342
loss=0.40193
loss=0.21293
loss=0.15496
loss=0.36740
loss=0.32209
loss=0.55274
loss=0.73862
loss=0.08605
loss=0.17305
loss=0.05489
loss=0.18451
loss=0.42559
loss=0.17030
loss=0.38825
loss=0.61447
loss=0.52366
loss=0.15975
loss=0.19142
loss=0.14575
loss=0.38191
loss=0.55312
loss=0.22514
loss=0.06152
loss=0.46176
loss=0.30120
loss=0.05278
loss=0.25801
loss=0.08760
loss=0.17734
loss=0.10760
loss=0.37058
loss=0.99459
loss=0.29430
loss=0.04090
loss=0.13038
loss=0.17189
loss=0.02757
loss=0.10815
loss=0.01692
loss=0.51303
loss=0.35512
loss=0.17924
loss=0.09479
loss=0.10803
loss=0.26539
loss=0.30867
loss=0.11393
loss=0.58416
loss=0.47937
loss=0.11339
loss=0.34520
loss=0.17211
loss=0.19526
loss=0.15832
loss=0.07887
loss=0.25074
